In [72]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
import os
import seaborn as sns
import tensorflow as tf
import os as os
import cv2
import imghdr
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import torch 
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as trans
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.layers.normalization import BatchNormalization
%matplotlib

ModuleNotFoundError: No module named 'tensorflow.layers'

In [69]:
SIZE = 128

In [37]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [38]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [39]:
tf.config.list_physical_devices('GPU')

[]

In [40]:
gpus = tf.config.experimental.list_physical_devices('CPU')

In [41]:
gpus_local = tf.config.experimental.list_logical_devices('GPU')

In [42]:
len(gpus)

1

In [43]:
len(gpus_local)

0

In [60]:
print(os.listdir("data"))

['Test', 'Train']


In [44]:
data_dir = 'data'

In [45]:
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [46]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print('Issue with image {}'.format(image_path))
             #os.remove("data")

In [47]:
data = tf.keras.utils.image_dataset_from_directory('data')

Found 442 files belonging to 2 classes.


In [48]:
data_iterator = data.as_numpy_iterator()

In [49]:
batch = data_iterator.next()

In [50]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

In [51]:
data = data.map(lambda x,y: (x/255, y))

In [52]:
data.as_numpy_iterator().next()

(array([[[[4.83448952e-01, 3.91822726e-01, 1.17647059e-01],
          [5.21943688e-01, 5.48823595e-01, 1.88897103e-01],
          [7.13737726e-01, 6.03137195e-01, 2.49675199e-01],
          ...,
          [6.48984909e-01, 5.42673647e-01, 2.50731230e-01],
          [6.75796568e-01, 5.69485307e-01, 2.77542889e-01],
          [6.51341081e-01, 5.33265114e-01, 2.69461602e-01]],
 
         [[5.46968102e-01, 4.87500012e-01, 1.65615186e-01],
          [5.19440293e-01, 5.05594671e-01, 1.81880221e-01],
          [7.68063784e-01, 6.31390989e-01, 3.70876163e-01],
          ...,
          [6.23027027e-01, 5.32830954e-01, 2.26948574e-01],
          [6.43137276e-01, 5.52941203e-01, 2.47058824e-01],
          [6.37387276e-01, 5.36743879e-01, 2.28196084e-01]],
 
         [[5.53302586e-01, 5.21270394e-01, 1.56323478e-01],
          [5.67781568e-01, 4.83363718e-01, 2.25410253e-01],
          [7.25245357e-01, 5.47437012e-01, 4.23784792e-01],
          ...,
          [6.52216196e-01, 5.41002929e-01, 2.3297

In [53]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [54]:
val_size

2

In [55]:
train_size

9

In [56]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)
model = Sequential()
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [57]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 16)        4624      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 16)       0

In [58]:
logdir='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

Epoch 1/20
9/9 [==============================] - 9s 840ms/step - loss: 0.9370 - accuracy: 0.6007 - val_loss: 0.8272 - val_accuracy: 0.5469
Epoch 2/20
9/9 [==============================] - 9s 881ms/step - loss: 0.6936 - accuracy: 0.5799 - val_loss: 0.6534 - val_accuracy: 0.5938
Epoch 3/20
9/9 [==============================] - 9s 950ms/step - loss: 0.6562 - accuracy: 0.6111 - val_loss: 0.6384 - val_accuracy: 0.6250
Epoch 4/20
9/9 [==============================] - 9s 875ms/step - loss: 0.6284 - accuracy: 0.6146 - val_loss: 0.6199 - val_accuracy: 0.6562
Epoch 5/20
9/9 [==============================] - 8s 769ms/step - loss: 0.6097 - accuracy: 0.6528 - val_loss: 0.6954 - val_accuracy: 0.5625
Epoch 6/20
9/9 [==============================] - 7s 763ms/step - loss: 0.5872 - accuracy: 0.6424 - val_loss: 0.4937 - val_accuracy: 0.6875
Epoch 7/20
9/9 [==============================] - 8s 792ms/step - loss: 0.5498 - accuracy: 0.7049 - val_loss: 0.5193 - val_accuracy: 0.7031
Epoch 8/20
9/9 [====

In [ ]:
prediction_NN = model.predict(train_size)
prediction_NN = np.argmax(prediction_NN, axis=-1)
prediction_NN = le.inverse_transform(prediction_NN)

In [ ]:
#plot the training and validation accuracy and loss at each epoch
loss = hist.history['loss']
val_loss = hist.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
#Confusion Matrix - verify accuracy of each class
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_size, prediction_NN)
print(cm)
sns.heatmap(cm, annot=True)

In [61]:
train_images = []
train_labels = [] 
for directory_path in glob.glob("data/train/*"):
    label = directory_path.split("\\")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)
        
train_images = np.array(train_images)
train_labels = np.array(train_labels)

1-s2.0-S1319562X12000332-gr5 - Copy.jpg
1-s2.0-S1319562X12000332-gr5.jpg
15-investigatin - Copy.jpg
15-investigatin.jpg
315748_1_En_8_Fig2_HTML - Copy.jpg
315748_1_En_8_Fig2_HTML.jpg
aam-aaes80642 - Copy.jpg
aam-aaes80642.jpg
algae-for-water-treatment-system-2-320 - Copy.jpg
algae-for-water-treatment-system-2-320.jpg
Algae-typically-found-in-facultative-ponds-a-Chlamydomonas-b-Chlorella-c_Q320 - Copy.jpg
Algae-typically-found-in-facultative-ponds-a-Chlamydomonas-b-Chlorella-c_Q320.jpg
Australian-algae-producer-enters-animal-feed-market - Copy.jpg
Australian-algae-producer-enters-animal-feed-market.jpg
d-various-rod-shaped-bacteria-magnification-2000x-at-35mm-image-size-2BE0GTE - Copy.jpg
d-various-rod-shaped-bacteria-magnification-2000x-at-35mm-image-size-2BE0GTE.jpg
eg3 - Copy.jpg
eg3.jpg
euglena - Copy.jpg
euglena-2GRMTJ2 - Copy.jpg
euglena-2GRMTJ2.jpg
euglena-dsc_0169 - Copy.jpg
euglena-dsc_0169.jpg
EUGLENA-labels-600 - Copy.jpg
EUGLENA-labels-600.jpg
euglena-with-flagella-captioned

In [63]:
# test
test_images = []
test_labels = [] 
for directory_path in glob.glob("data/test/*"):
    fruit_label = directory_path.split("\\")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        test_images.append(img)
        test_labels.append(fruit_label)
        
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [64]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

In [65]:
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

In [66]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [70]:
activation = 'sigmoid'

feature_extractor = Sequential()
feature_extractor.add(Conv2D(32, 3, activation = activation, padding = 'same', input_shape = (SIZE, SIZE, 3)))
feature_extractor.add(BatchNormalization())

feature_extractor.add(Conv2D(32, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
feature_extractor.add(BatchNormalization())
feature_extractor.add(MaxPooling2D())

feature_extractor.add(Conv2D(64, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
feature_extractor.add(BatchNormalization())

feature_extractor.add(Conv2D(64, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
feature_extractor.add(BatchNormalization())
feature_extractor.add(MaxPooling2D())

feature_extractor.add(Flatten())

NameError: name 'BatchNormalization' is not defined